In [65]:
import os
import boto3
from braket.aws import AwsQuantumJob
from braket.jobs.config import OutputDataConfig, CheckpointConfig

from datetime import datetime

region = "us-west-2"  
account_id = boto3.client("sts").get_caller_identity()["Account"]
s3_bucket = f"amazon-braket-us-west-2-{account_id}"

print("AWS Account:", account_id)
print("S3 Bucket:", s3_bucket)


AWS Account: 381305464630
S3 Bucket: amazon-braket-us-west-2-381305464630


In [66]:
!ls -lh build


total 40K
-rw-r--r--. 1 ec2-user ec2-user 835 Nov 30 05:53 qubo_sparse.csv
-rw-r--r--. 1 ec2-user ec2-user 15K Nov 30 04:57 sections.json
-rw-r--r--. 1 ec2-user ec2-user 20K Nov 30 04:57 slots.json


In [67]:
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
s3_prefix = f"jobs/schedule-qaoa/{timestamp}/input"

s3_client = boto3.client("s3")

def upload(path):
    filename = os.path.basename(path)
    key = f"{s3_prefix}/{filename}"
    s3_client.upload_file(path, s3_bucket, key)
    print("Uploaded:", key)
    return f"s3://{s3_bucket}/{key}"

qubo_s3 = upload("build/qubo_sparse.csv")
sections_s3 = upload("build/sections.json")
slots_s3 = upload("build/slots.json")


Uploaded: jobs/schedule-qaoa/20251130-070129/input/qubo_sparse.csv
Uploaded: jobs/schedule-qaoa/20251130-070129/input/sections.json
Uploaded: jobs/schedule-qaoa/20251130-070129/input/slots.json


In [68]:
!ls -lh src/hybrid_runner.py


-rw-r--r--. 1 ec2-user ec2-user 3.7K Nov 30 06:07 src/hybrid_runner.py


In [69]:
job_name = f"qaoa-tn1-small-{timestamp}"

print("Submitting job:", job_name)


Submitting job: qaoa-tn1-small-20251130-070129


In [70]:
job = AwsQuantumJob.create(
    # TARGET: TN1 tensor-network simulator
    device="arn:aws:braket:::device/quantum-simulator/amazon/tn1",

    # Module containing hybrid_runner.py
    source_module="src",

    # Entry point function
    entry_point="src.hybrid_runner:run",

    # Input channel: must be a *string* S3 URI, not a dict
    input_data={
        "input": f"s3://{s3_bucket}/{s3_prefix}/"
    },

    # Hyperparameters forwarded to run(**kwargs)
    hyperparameters={
        "p": 1,
        "shots": 500,
    },

    # Where job results go
    output_data_config=OutputDataConfig(
        s3Path=f"s3://{s3_bucket}/jobs/{job_name}/output"
    ),

    # Optional checkpoints
    checkpoint_config=CheckpointConfig(
        s3Uri=f"s3://{s3_bucket}/jobs/{job_name}/checkpoints"
    ),

    wait_until_complete=False,
)

print("Job ARN:", job.arn)


Job ARN: arn:aws:braket:us-west-2:381305464630:job/aa634bf1-1804-404f-aef7-fee9698cfc1d


In [73]:
print("Job state:", job.state())

Job state: COMPLETED


In [74]:
results = job.result()
results

{'counts': {'00011111101100011111100100011000': 1,
  '10011001100100010101000011100101': 1,
  '01001101101111010101101000011011': 1,
  '00101001101111001100101000111001': 1,
  '00101001011111111110110010111011': 1,
  '01000110001110101011111001001100': 1,
  '10001110101110010010000001111111': 1,
  '10111110101111011001100011110011': 1,
  '11001010000011100010001001110010': 1,
  '01111010100110110100001011011011': 1,
  '11101110101111100000001010000000': 1,
  '00011001111001000100101001000111': 1,
  '10011010011001001110000110011110': 1,
  '01101110110011100001001011111100': 1,
  '10101110110100000000011011000010': 1,
  '11111111011110101000100110000010': 1,
  '10010110001110101111011110000001': 1,
  '00001101101000010000000001000101': 1,
  '11001111110011110010110000000010': 1,
  '00110111000011001010111000001111': 1,
  '11001110111101111001011011100000': 1,
  '01100000000011001010100111010101': 1,
  '01101100110001110011010011110111': 1,
  '01100010001110111100000000110000': 1,
  '010